# Import the libraries

In [1]:
import pandas as pd
from linearmodels import PanelOLS
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Set the font to Times New Roman
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'

# Prepare the data

In [2]:
ind_col = 'naicsh6'
nlp_model = 'mpnet'
doc2vec_vector_size = 768 # 768 for mpnet, 1536 for ada

data = pd.read_pickle(f'../data/all_reg_vars_{nlp_model}_{str(doc2vec_vector_size)}_{ind_col}.pkl')
data.replace([np.inf, -np.inf], np.nan, inplace=True)


### Create lead variables for the DVs

In [3]:
# Sort data by GVKEY and year
reg_data = data.sort_values(by=['GVKEY', 'year'])
# Lead dependent variables by one year
reg_data['numrec_mean_0_lead'] = reg_data.groupby('GVKEY')['numrec_mean_0'].shift(-1)
reg_data['meanrec_mean_descend_lead'] = reg_data.groupby('GVKEY')['meanrec_mean_descend'].shift(-1)
reg_data['year_lead'] = reg_data.groupby('GVKEY')['year'].shift(-1)
reg_data['year_diff'] = reg_data['year_lead'] - reg_data['year']
reg_data = reg_data[reg_data['year_diff'] <=2 ]
reg_data['constant'] = 1

In [4]:
# Limit the data to non-exemplar firms and industries with at least 10 firms
reg_data = reg_data[(reg_data[f'exemplar_{ind_col}'] == 0) & (reg_data[f'no_firms_{ind_col}'] > 9)]

In [5]:
reg_data['year_lead'] = reg_data.groupby('GVKEY')['year'].shift(-1)

### Rename the variables to make them more readable

In [6]:
# define a dictionary that maps old names to new names
rename_dict = {
    'numrec_mean_0_lead': 'Analyst Coverage',
    'meanrec_mean_descend_lead': 'Analyst Recoms',
    f'exemplar_sim_{ind_col}': 'Exemplar Similarity',
    'sale_wins_1': 'Total Sales',
    'n_emp': 'Firm Size',
    'EPS_wins_1': 'EPS',
    'slack_avail_wins_1': 'Available Slack',
    'rd_f_wins_1': 'R&D Expenditure',
    'adv_f_wins_1': 'Advertising Expenditure',
    'dpt_f_wins_1': 'Depreciation Ratio',
    'intang_f_wins_1': 'Intangible Assets Ratio',
    'n_segments': 'No. Segments',
    f'market_share_{ind_col}': 'Market Share',
    'mergers_wins_1': 'Mergers (expenditure)',
    'leverage_wins_1': 'Financial Leverage',
    'is_spx500': 'S&P500 Dummy',
    f'sim_mean_{ind_col}': 'Firm Typicality',
    'strong_weak_modal_ratio': 'Strong-Weak Modals Ratio',
    'positive_negative_ratio': 'Positive-Negative Words Ratio',
    'N_Litigious': 'Litigous Words Ratio',
    f'n_analysts_{ind_col}': 'No. Analysts in Industry',
    f'no_firms_{ind_col}': 'No. Firms in Industry',
    f'category_coherence_{ind_col}': 'Category Coherence',
    f'numrec_avg_year_{ind_col}': 'Average Coverage (Year-Ind)',
    f'meanrec_avg_year_{ind_col}': 'Average Recoms (Year-Ind)',
    f'ind_sim_all_{ind_col}_distinct': 'Category Distinctiveness',
    f'ind_vecs_change_{ind_col}': 'Category Instability',
    f'hhi_{ind_col}': 'Industry HHI',
    f'EPS_wins_1_ex_{ind_col}': 'Exemplar EPS',
    f'exemplar_{ind_col}_typicality': 'Exemplar Typicality',
    'strategy_unique': 'Strategy Uniqueness',
    'sim_10nearest' : '10-Nearest Similarity',
    'firm_average_analyst_coverage' : 'Analyst Average Portfolio Size',
    'firm_complexity' : 'Firm Complexity',
    'positive_negative_ratio': 'Positive-Negative Words Ratio',
    'strong_weak_modal_ratio' : 'Strong-Weak Modals Ratio',
    'N_Uncertainty' : 'Uncertainty Words Ratio',
    'N_Litigious' : 'Litigious Words Ratio',
    'constant': 'Constant'
}

# rename columns
reg_data = reg_data.rename(columns=rename_dict)


In [7]:
reg_data['Analyst Average Portfolio Size'].describe()

count    41801.000000
mean        18.161131
std          9.022695
min          1.000000
25%         13.600000
50%         17.400000
75%         21.750000
max        203.000000
Name: Analyst Average Portfolio Size, dtype: float64

In [112]:
# replace missing values of word ratios with mean to avoid losing observations
reg_data['Positive-Negative Words Ratio'] = reg_data['Positive-Negative Words Ratio'].fillna(reg_data['Positive-Negative Words Ratio'].mean())
reg_data['Strong-Weak Modals Ratio'] = reg_data['Strong-Weak Modals Ratio'].fillna(reg_data['Strong-Weak Modals Ratio'].mean())
reg_data['Uncertainty Words Ratio'] = reg_data['Uncertainty Words Ratio'].fillna(reg_data['Uncertainty Words Ratio'].mean())
reg_data['Litigious Words Ratio'] = reg_data['Litigious Words Ratio'].fillna(reg_data['Litigious Words Ratio'].mean())


In [113]:
print(reg_data['Strategy Uniqueness'].corr(reg_data['Exemplar Similarity']))
print(reg_data['Strategy Uniqueness'].corr(reg_data['Analyst Average Portfolio Size']))
print(reg_data['Firm Typicality'].corr(reg_data['Analyst Average Portfolio Size']))
print(reg_data['Exemplar Similarity'].corr(reg_data['Analyst Average Portfolio Size']))
print(reg_data['Firm Typicality'].corr(reg_data['Exemplar Similarity']))
print(reg_data['10-Nearest Similarity'].corr(reg_data['Exemplar Similarity']))
print(reg_data['Firm Typicality'].corr(reg_data['Firm Complexity']))

-0.1074931704852451
-0.035294162052742196
0.11670300609786073
0.16977470020910101
0.7476796139665128
0.7882235176243836
-0.009676309736095625


### Define the list of independent/control variables


In [114]:
iv = ['Exemplar Similarity']
moderators = ['Category Coherence', 'Category Distinctiveness','Exemplar Typicality']
controls = [
            'Total Sales', 'Firm Size','Market Share', 'EPS', 'Available Slack', 'R&D Expenditure',
            'Advertising Expenditure', 'Intangible Assets Ratio', 'Depreciation Ratio', 
              'No. Segments',
             'Mergers (expenditure)', 'Financial Leverage', 'S&P500 Dummy', 
             'No. Analysts in Industry', 'No. Firms in Industry',  
             'Average Coverage (Year-Ind)', 'Average Recoms (Year-Ind)',
             'Category Instability', 'Industry HHI', 'Exemplar EPS',
             'Constant']
dv = ['Analyst Coverage', 'Analyst Recoms']

In [115]:
# Set index to GVKEY and year
reg_data = reg_data.set_index(['GVKEY', 'year'])

# Regression Models

## Effect of Exemplar similarity on Analyst Coverage

In [116]:
regressors = iv + moderators + controls
model1 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res1 = model1.fit(cov_type='clustered', cluster_entity=True)
res1


Dep. Variable:,Analyst Coverage,R-squared:,0.1184
Estimator:,PanelOLS,R-squared (Between):,0.2010
No. Observations:,46786,R-squared (Within):,0.1184
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.1779
Time:,16:05:15,Log-likelihood,-1.06e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,219.15
Entities:,7603,P-value,0.0000
Avg Obs:,6.1536,Distribution:,"F(24,39159)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,34.553


## Effect of Exemplar Similarity on Analyst Recommendations

In [117]:
regressors = iv + moderators + controls
model2 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res2 = model2.fit(cov_type='clustered', cluster_entity=True)
res2

Dep. Variable:,Analyst Recoms,R-squared:,0.0526
Estimator:,PanelOLS,R-squared (Between):,0.0297
No. Observations:,30688,R-squared (Within):,0.0526
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0410
Time:,16:05:16,Log-likelihood,-1.268e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,57.081
Entities:,5975,P-value,0.0000
Avg Obs:,5.1361,Distribution:,"F(24,24689)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,29.304


## Moderation Effect of Category Coherence

In [118]:
# Regressions with interaction terms - analyst coverage and category coherence
interaction_term = 'Exemplar Similarity x Category Coherence'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Coherence']
regressors = iv + moderators + [interaction_term] + controls

model3 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res3 = model3.fit(cov_type='clustered', cluster_entity=True)
res3


Dep. Variable:,Analyst Coverage,R-squared:,0.1214
Estimator:,PanelOLS,R-squared (Between):,0.1940
No. Observations:,46786,R-squared (Within):,0.1214
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.1703
Time:,16:05:16,Log-likelihood,-1.059e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,216.42
Entities:,7603,P-value,0.0000
Avg Obs:,6.1536,Distribution:,"F(25,39158)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,34.638


In [119]:
interaction_term = 'Exemplar Similarity x Category Coherence'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Coherence']
regressors = iv + moderators + [interaction_term] + controls

model4 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res4 = model4.fit(cov_type='clustered', cluster_entity=True)
res4

Dep. Variable:,Analyst Recoms,R-squared:,0.0529
Estimator:,PanelOLS,R-squared (Between):,0.0327
No. Observations:,30688,R-squared (Within):,0.0529
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0450
Time:,16:05:16,Log-likelihood,-1.267e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,55.116
Entities:,5975,P-value,0.0000
Avg Obs:,5.1361,Distribution:,"F(25,24688)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,28.667


## Moderation Effect of Category Distinctiveness

In [120]:
# Regressions with interaction terms - analyst coverage and category coherence
interaction_term = 'Exemplar Similarity x Category Distinctiveness'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Distinctiveness']
regressors = iv + moderators + [interaction_term] + controls

model5 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res5 = model5.fit(cov_type='clustered', cluster_entity=True)
res5


Dep. Variable:,Analyst Coverage,R-squared:,0.1206
Estimator:,PanelOLS,R-squared (Between):,0.2048
No. Observations:,46786,R-squared (Within):,0.1206
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.1810
Time:,16:05:17,Log-likelihood,-1.059e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,214.72
Entities:,7603,P-value,0.0000
Avg Obs:,6.1536,Distribution:,"F(25,39158)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,34.273


In [121]:
interaction_term = 'Exemplar Similarity x Category Distinctiveness'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Category Distinctiveness']
regressors = iv + moderators + [interaction_term] + controls

model6 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res6 = model6.fit(cov_type='clustered', cluster_entity=True)
res6

Dep. Variable:,Analyst Recoms,R-squared:,0.0530
Estimator:,PanelOLS,R-squared (Between):,0.0188
No. Observations:,30688,R-squared (Within):,0.0530
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0341
Time:,16:05:17,Log-likelihood,-1.267e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,55.256
Entities:,5975,P-value,0.0000
Avg Obs:,5.1361,Distribution:,"F(25,24688)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,28.409


## Moderation Effect of Exemplar Typicality

In [122]:
# Regressions with interaction terms - analyst coverage and category coherence
interaction_term = 'Exemplar Similarity x Exemplar Typicality'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Exemplar Typicality']
regressors = iv + moderators + [interaction_term] + controls

model7 = PanelOLS(reg_data['Analyst Coverage'], reg_data[regressors], entity_effects=True)
res7 = model7.fit(cov_type='clustered', cluster_entity=True)
res7

Dep. Variable:,Analyst Coverage,R-squared:,0.1194
Estimator:,PanelOLS,R-squared (Between):,0.1996
No. Observations:,46786,R-squared (Within):,0.1194
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.1762
Time:,16:05:17,Log-likelihood,-1.059e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,212.41
Entities:,7603,P-value,0.0000
Avg Obs:,6.1536,Distribution:,"F(25,39158)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,33.941


In [123]:
interaction_term = 'Exemplar Similarity x Exemplar Typicality'
reg_data[interaction_term] = reg_data['Exemplar Similarity'] * reg_data['Exemplar Typicality']
regressors = iv + moderators + [interaction_term] + controls

model8 = PanelOLS(reg_data['Analyst Recoms'], reg_data[regressors], entity_effects=True)
res8 = model8.fit(cov_type='clustered', cluster_entity=True)
res8

Dep. Variable:,Analyst Recoms,R-squared:,0.0532
Estimator:,PanelOLS,R-squared (Between):,0.0320
No. Observations:,30688,R-squared (Within):,0.0532
Date:,"Tue, Jun 20 2023",R-squared (Overall):,0.0442
Time:,16:05:18,Log-likelihood,-1.267e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,55.450
Entities:,5975,P-value,0.0000
Avg Obs:,5.1361,Distribution:,"F(25,24688)"
Min Obs:,1.0000,,
Max Obs:,25.000,F-statistic (robust):,28.780


# Output regression results

In [124]:
def create_table(*results):
    table = pd.DataFrame()
    for i, result in enumerate(results, start=1):
        coefficients = result.params
        pvalues = result.pvalues
        nobs = result.nobs
        rsquared = "{:.3f}".format(result.rsquared)
        nentities = result._entity_info['total']

        # Create a DataFrame for this result
        data = {}
        for key in coefficients.keys():
            data[key] = ["{:.3f}".format(coefficients[key])]
            data[key + ' p-val'] = ["'({:.3f})".format(pvalues[key])]  # added single quote
        data['Observations'] = [nobs]
        data['R-squared'] = [rsquared]
        data['Entities'] = [nentities]
        df = pd.DataFrame(data)

        # Assign model name to the index
        df.index = ['Model' + str(i)]

        # Concatenate to the final table
        table = pd.concat([table, df], axis=0)
    return table

In [125]:

# Now create the table with your results:
table = create_table(res1, res2, res3, res4, res5, res6, res7, res8)


In [126]:
table.T.to_csv(f'../data/tables/main_regression_results_notypicality_control.csv')